In [13]:
import torch
import csv
import pandas as pd

In [14]:
# Input NFT collection names and their floor prices, market volume
def nft_market_data(nft_name):
    df1 = pd.read_csv(nft_name +'_floor_price.csv')
    df2 = pd.read_csv(nft_name +'_market_volume.csv')
    market_volume = df2['market_volume'].tolist()
    floor_price = df1['floor_price'].tolist()
    
    floor_price.pop()
    market_volume.pop()
    return floor_price, market_volume

In [15]:
import os
import pandas as pd
import numpy as np

def nft_vadar_sentiment_score(nft_name):
    # Read in the vader folder and get all 
    nft_vadar_sentiment_score = []
    start_date = '2022-08-01'
    end_date = '2023-02-27'
    date_range = pd.date_range(start_date, end_date)
    
    for date in date_range:
        date_str = date.strftime('%Y-%m-%d')
        csv_file = 'vader/' + 'vader' + '_' + date_str + '.csv'
        # filter the Text column and get all the indices
        df = pd.read_csv(csv_file)
        # keep only rows whose 'Text' has the nft_name
        df = df[df['Text'].str.contains(nft_name)]
        # if there is no row with the nft_name, then the vader score is 0
        if df.empty:
            nft_vadar_sentiment_score.append(0)
            continue
        # get the vader column's compound score
        vader_col = df['vader']
        # extract the compound score
        # Extract the compound number from each row of the 'vader' column
        compound_list = [eval(vader)['compound'] for vader in vader_col]
        # calculate the average compound score
        vader_avg = np.mean(compound_list)
        nft_vadar_sentiment_score.append(vader_avg)
    return nft_vadar_sentiment_score


In [9]:
# test the function
azuki_sentiment_scores = nft_vadar_sentiment_score('azuki')

In [10]:
# print azuki_sentiment_scores
print(azuki_sentiment_scores)

[0.20656249999999998, 0.765, 0.2202, -0.29569999999999996, 0.0371, 0.40519999999999995, 0.37649999999999995, 0.4404, -0.0258, 0.6065999999999999, 0.49888000000000005, 0.0, 0.0, -0.3612, 0.15766, 0.07587142857142858, 0.660225, 0.346, 0.2171833333333333, 0.14965, 0, 0.6369, -0.296, 0.5137333333333333, 0.19191428571428573, 0.29957500000000004, 0.048725000000000004, 0, 0.0, -0.20095, -0.3306, 0.031657142857142856, 0.1752166666666667, 0.0, 0.5189, 0.11011666666666665, 0.0, -0.8225, 0.079, 0.0772, 0.4215, 0.26353333333333334, 0.167625, 0.0, 0.229075, 0.12688000000000002, 0.33125, -0.32926666666666665, 0.421, 0.10366250000000002, 0.0277, -0.2352, 0.07876249999999999, 0.025733333333333334, 0.0, 0.45690000000000003, -0.0086, 0.09658333333333334, 0.20230909090909088, 0.0772, -0.045936842105263165, -0.20380000000000004, 0.2294, 0.4141333333333333, 0.2587, 0.5102, -0.015849999999999975, 0.28501999999999994, 0.5445666666666668, -0.0172, 0.21715000000000004, 0.2454555555555556, 0.14702727272727276, 

In [7]:
# print the sentiment scores
# fill the missing values with 0
azuki_sentiment_scores = [0 if np.isnan(x) else x for x in azuki_sentiment_scores]

In [16]:
import os
import pandas as pd
import numpy as np

def nft_tweets_textual_embeddings(nft_name, embedding_name):
    average_embeddings = []

    start_date = '2022-08-01'
    end_date = '2023-02-27'
    date_range = pd.date_range(start_date, end_date)
    sentiment_score = nft_vadar_sentiment_score(nft_name)
    for date in date_range:
        date_str = date.strftime("%Y-%m-%d")
        csv_file = os.path.join(f"{date_str}.csv")
        embeddings = []
        if os.path.exists(csv_file):
            df = pd.read_csv(csv_file)
            keyword_indices = df[df['Text'].str.contains(nft_name, case=False)].index

            daily_embedding_folder = os.path.join(f"/home/qian/qian/CS6220-NFT-Celebrities-Twitter-Analysis-main/{embedding_name}/{date_str}")
            for index in keyword_indices:
                index_file = os.path.join(f"{daily_embedding_folder}/{index}.npy")
                if os.path.exists(index_file):
                    # keep only the first 768 dimensions
                    embedding = np.load(index_file)[..., :768]
                    embedding_dim = embedding.shape[0]
                    embeddings.append(embedding)

            if len(embeddings) > 0:
                average_bert_embeddings = np.mean(embeddings, axis=0)
                # then append vader sentiment score the average bert embeddings, 
                average_bert_embeddings = np.append(average_bert_embeddings, )
            else:
                average_bert_embeddings = np.zeros((768,))
            average_embeddings.append(average_bert_embeddings)
    return average_embeddings


In [270]:
# test:nft_market_data('azuki')
azuki_price, azuki_volume = nft_market_data('azuki')

In [271]:
# convert list to tensor
azuki_price_tensor = torch.tensor(azuki_price, dtype=torch.float32)
azuki_volume_tensor = torch.tensor(azuki_volume, dtype=torch.float32)

In [280]:
# get azuki textual embeddings and convert to tensor
azuki_embeddings = nft_tweets_textual_embeddings('azuki', 'bert_embeddings')
azuki_embeddings_tensor = torch.tensor(azuki_embeddings, dtype=torch.float32)

In [17]:
# get bayc data and convert to tensor
bayc_price, bayc_volume = nft_market_data('bayc')
bayc_price_tensor = torch.tensor(bayc_price, dtype=torch.float32)
bayc_volume_tensor = torch.tensor(bayc_volume, dtype=torch.float32)
bayc_embeddings = nft_tweets_textual_embeddings('bayc', 'bert_embeddings')
bayc_embeddings_tensor = torch.tensor(bayc_embeddings, dtype=torch.float32)

/tmp/ipykernel_846733/3839738689.py:6: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  /home/conda/feedstock_root/build_artifacts/pytorch-recipe_1660070785140/work/torch/csrc/utils/tensor_new.cpp:201.)
  bayc_embeddings_tensor = torch.tensor(bayc_embeddings, dtype=torch.float32)


In [18]:
# mayc data and convert to tensor
mayc_price, mayc_volume = nft_market_data('mayc')
mayc_price_tensor = torch.tensor(mayc_price, dtype=torch.float32)
mayc_volume_tensor = torch.tensor(mayc_volume, dtype=torch.float32)
mayc_embeddings = nft_tweets_textual_embeddings('mayc', 'bert_embeddings')
mayc_embeddings_tensor = torch.tensor(mayc_embeddings, dtype=torch.float32)

In [19]:
# print mayc data
mayc_embeddings_tensor[:5]

tensor([[-0.2123, -0.0152,  0.0763,  ..., -0.3050,  0.2155,  0.3054],
        [-0.1550, -0.0228,  0.1331,  ..., -0.2174,  0.2856,  0.3664],
        [-0.1934,  0.0528,  0.0591,  ..., -0.2550,  0.2446,  0.3307],
        [-0.2766, -0.0902,  0.1316,  ..., -0.3048,  0.2624,  0.4160],
        [-0.1113,  0.0855,  0.2011,  ..., -0.2698,  0.2867,  0.2766]])

In [20]:
# otherdeed data and convert to tensor
otherdeed_price, otherdeed_volume = nft_market_data('otherdeed_for_otherside')
otherdeed_price_tensor = torch.tensor(otherdeed_price, dtype=torch.float32)
otherdeed_volume_tensor = torch.tensor(otherdeed_volume, dtype=torch.float32)
otherdeed_embeddings = nft_tweets_textual_embeddings('otherdeed', 'bert_embeddings')
otherdeed_embeddings_tensor = torch.tensor(otherdeed_embeddings, dtype=torch.float32)

In [21]:
# clonex data and convert to tensor
clonex_price, clonex_volume = nft_market_data('clonex')
clonex_price_tensor = torch.tensor(clonex_price, dtype=torch.float32)
clonex_volume_tensor = torch.tensor(clonex_volume, dtype=torch.float32)
clonex_embeddings = nft_tweets_textual_embeddings('clonex', 'bert_embeddings')
clonex_embeddings_tensor = torch.tensor(clonex_embeddings, dtype=torch.float32)

In [22]:
# print azuki_embeddings_tensor last 5 rows
print(azuki_embeddings_tensor[:5])

NameError: name 'azuki_embeddings_tensor' is not defined

In [42]:
# Train LSTM model
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import math

# Prepare dataset
X = clonex_embeddings_tensor  # Shape: (number_of_dates, 768)
y = clonex_price_tensor # Shape: (number_of_dates, 1)

# Reshape the data before normalization
X_numpy = X.numpy()
y_numpy = y.numpy().reshape(-1, 1)

# Split the dataset into training and test sets, use first 80% of the data for training
X_train, X_test, y_train, y_test = train_test_split(X_numpy, y_numpy, test_size=0.2, shuffle=False)

# Normalize the input data
scaler_X = MinMaxScaler()
X_train_normalized = scaler_X.fit_transform(X_train)
X_test_normalized = scaler_X.transform(X_test)

scaler_y = MinMaxScaler()
y_train_normalized = scaler_y.fit_transform(y_train)

# Convert the normalized data back to tensors
X_train = torch.tensor(X_train_normalized, dtype=torch.float32)
X_test = torch.tensor(X_test_normalized, dtype=torch.float32)
y_train = torch.tensor(y_train_normalized, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.float32)

# Create DataLoader
train_dataset = TensorDataset(X_train, y_train)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

# Define the LSTM model architecture
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size, dropout_prob=0.2):
        super(LSTMModel, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.dropout = nn.Dropout(dropout_prob)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size)
        out, _ = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])
        return out

# Initialize the model, loss function, and optimizer
input_size = 768
hidden_size = 32
num_layers = 2
output_size = 1
drop_prob = 0.1
learning_rate = 0.001

model = LSTMModel(input_size, hidden_size, num_layers, output_size, drop_prob)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Train the model with early stopping
num_epochs = 200
early_stop = 20 # Number of epochs to wait before early stopping
best_loss = float('inf')
best_epoch = 0
best_model = None

for epoch in range(num_epochs):
    for i, (embeddings, targets) in enumerate(train_loader):
        embeddings = embeddings.unsqueeze(1)  # Shape: (batch_size, 1, 770)

        # Forward pass
        outputs = model(embeddings)
        loss = criterion(outputs, targets)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    if loss.item() < best_loss:
        best_loss = loss.item()
        best_epoch = epoch
        best_model = model.state_dict()
    elif epoch - best_epoch >= early_stop:
        print(f'Early stopping at epoch {epoch+1}')
        break

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

# Evaluate the model on the test set
model.eval()

with torch.no_grad():
    X_test = X_test.unsqueeze(1)  # Shape: (test_size, 1, 770)
    y_pred_normalized = model(X_test)

    # Inverse transform the normalized predictions to get the unnormalized predictions
    y_pred = scaler_y.inverse_transform(y_pred_normalized.numpy())

# Calculate the mean squared error or any other performance metric
mse = criterion(torch.tensor(y_pred), y_test)
# keep 2 decimal places
# Round mean squared error to two decimal places
mse_rounded = torch.round(mse * 100) / 100

# Print mean squared error with two decimal places
print(f"Mean Squared Error: {mse_rounded:.2f}")

rmse = math.sqrt(mse.item())
print('Root Mean Squared Error:', rmse)

# Add some code to document the results


Epoch [1/200], Loss: 0.0640
Epoch [2/200], Loss: 0.0404
Epoch [3/200], Loss: 0.1220
Epoch [4/200], Loss: 0.0459
Epoch [5/200], Loss: 0.1161
Epoch [6/200], Loss: 0.1043
Epoch [7/200], Loss: 0.0914
Epoch [8/200], Loss: 0.0418
Epoch [9/200], Loss: 0.0516
Epoch [10/200], Loss: 0.0708
Epoch [11/200], Loss: 0.0927
Epoch [12/200], Loss: 0.0717
Epoch [13/200], Loss: 0.0477
Epoch [14/200], Loss: 0.0747
Epoch [15/200], Loss: 0.0739
Epoch [16/200], Loss: 0.0773
Epoch [17/200], Loss: 0.0482
Epoch [18/200], Loss: 0.1058
Epoch [19/200], Loss: 0.0830
Epoch [20/200], Loss: 0.0525
Epoch [21/200], Loss: 0.0994
Early stopping at epoch 22
Mean Squared Error: 4.86
Root Mean Squared Error: 2.2052378161631867
